# Vector Quantization (VQ) Layer Function

Related links:
* https://stats.stackexchange.com/questions/146221/is-cosine-similarity-identical-to-l2-normalized-euclidean-distance
* https://www.mathworks.com/help/stats/pdist.html
* https://stackoverflow.com/questions/37009647/compute-pairwise-distance-in-a-batch-without-replicating-tensor-in-tensorflow

In [1]:
import tensorflow as tf
from typing import List

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def vector_quantization(x: tf.Tensor, n, alpha=0.1, beta=1e-4, gamma=1e-6, embedding_initializer=tf.random_normal_initializer) -> tf.Tensor:
    # shape of x is [batch, , q], where this function quantizes along dimension q
    vec_size = x.shape[2]
    with tf.variable_scope('vq'):
        # embedding space
        emb_space = tf.get_variable('emb_space', shape=[n, vec_size], dtype=x.dtype, initializer=embedding_initializer, trainable=True)
        
        # map x to y, where y is the vector from emb_space that is closest to x
        diff = tf.expand_dims(x, axis=2) - emb_space  # distance of x from all vectors in the embedding space
        dist = tf.reduce_sum(diff ** 2, axis=2)  # l2 distance between x and all vectors in emb
        emb_index = tf.argmin(dist, axis=2)
        y = tf.gather(emb_space, emb_index, axis=0)

        # closest embedding update loss
        nearest_loss = alpha * tf.reduce_sum((y - x) ** 2, axis=1)
        tf.add_to_collection(tf.GraphKeys.LOSSES, nearest_loss)
        
        # all embeddings update loss
        all_loss = beta * tf.reduce_mean(dist, axis=2)
        tf.add_to_collection(tf.GraphKeys.LOSSES, all_loss)
        
        # all embeddings distance from each other (gamma)
        pdiff = tf.expand_dims(emb_space, axis=0) - tf.expand_dims(emb_space, axis=1)  # pair-wise diff vectors (n x n x vec_size)
        pdist = tf.reduce_sum(pdiff ** 2, axis=2)  # pair-wise L2 distance scalars (n x n)
        coulomb_loss = - gamma * tf.reduce_mean(pdist, axis=[0, 1])
        tf.add_to_collection(tf.GraphKeys.LOSSES, coulomb_loss)

        # return selection
        return tf.stop_gradient(y - x) + x  # skip this layer when doing back-prop

In [3]:
tf.reset_default_graph()
x = tf.placeholder(name='test', shape=[None, 5, 10], dtype=tf.float32)
a = vector_quantization(x, 200)

In [4]:
a

<tf.Tensor 'vq/add:0' shape=(?, 5, 10) dtype=float32>

In [5]:
e = tf.get_variable('emb_space', shape=[5, 200], dtype=tf.float32, initializer=tf.random_normal_initializer)

In [6]:
tf.expand_dims(e, axis=0) - tf.expand_dims(e, axis=1)

<tf.Tensor 'sub:0' shape=(5, 5, 200) dtype=float32>